# Model recommendation with lighfm

### Import libraries

In [ ]:
import os
import sys
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
import pandas as pd
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from lightfm import cross_validation
import scipy.sparse as sp
from scipy import sparse

In [ ]:
from lightfm import LightFM
from sklearn.base import clone


class LightFMResizable(LightFM):
    """A LightFM that resizes the model to accomodate new users,
    items, and features"""

    def fit_partial(
        self,
        interactions,
        user_features=None,
        item_features=None,
        sample_weight=None,
        epochs=1,
        num_threads=1,
        verbose=False,
    ):
        try:
            self._check_initialized()
            self._resize(interactions, user_features, item_features)
        except ValueError:
            # This is the first call so just fit without resizing
            pass

        super().fit_partial(
            interactions,
            user_features,
            item_features,
            sample_weight,
            epochs,
            num_threads,
            verbose,
        )

        return self

    def _resize(self, interactions, user_features=None, item_features=None):
        """Resizes the model to accommodate new users/items/features"""

        no_components = self.no_components
        no_user_features, no_item_features = interactions.shape  # default

        if hasattr(user_features, "shape"):
            no_user_features = user_features.shape[-1]
        if hasattr(item_features, "shape"):
            no_item_features = item_features.shape[-1]

        if (
            no_user_features == self.user_embeddings.shape[0]
            and no_item_features == self.item_embeddings.shape[0]
        ):
            return self

        new_model = clone(self)
        new_model._initialize(no_components, no_item_features, no_user_features)

        # update all attributes from self._check_initialized
        for attr in (
            "item_embeddings",
            "item_embedding_gradients",
            "item_embedding_momentum",
            "item_biases",
            "item_bias_gradients",
            "item_bias_momentum",
            "user_embeddings",
            "user_embedding_gradients",
            "user_embedding_momentum",
            "user_biases",
            "user_bias_gradients",
            "user_bias_momentum",
        ):
            # extend attribute matrices with new rows/cols from
            # freshly initialized model with right shape
            old_array = getattr(self, attr)
            old_slice = [slice(None, i) for i in old_array.shape]
            new_array = getattr(new_model, attr)
            new_array[tuple(old_slice)] = old_array
            setattr(self, attr, new_array)

        return self

### Defining variables

In [ ]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

In [ ]:
# NUM_ITEMS_TRAIN = 108573
NUM_ITEMS_TRAIN = 63893
# NUM_ITEMS_TRAIN = 231305

NO_EPOCHS = 80

NUM_COMPONENTS = 30

NUM_THREADS = 4

### Retrieve data

In [ ]:
dtype_df_valid = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"timestampHistory" : 'string'
}

In [ ]:
import pandas as pd
# path config

df_valid = pd.read_csv(config["VALID_DF"],dtype=dtype_df_valid,nrows=10000)
df_valid.head(3)

In [ ]:
dtype_df_train_score = {
"userId" : 'string',
"userType" : 'category',
"history" : 'string',
"score" : 'Float32'
}

In [ ]:
import pandas as pd

# df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score)
df_ratings = pd.read_csv(config["DF_TRAIN_SCORES"], dtype=dtype_df_train_score, nrows=500010)
df_ratings.drop(columns=["Unnamed: 0"],inplace=True)
df_ratings.head(3)

In [ ]:
df_news = pd.read_csv(config["DF_ITEMS_FEATURE"])
df_news.drop(columns=["Unnamed: 0"],inplace=True)
df_news.head(3)

In [ ]:
df_valid_new = df_valid.drop(columns="timestampHistory")

from utils.custom_treat_data_funcs import transform_text_to_list, explode_df_columns

# df_valid_new["history"] = df_valid_new["history"].apply(str)
df_valid_new["history"] = df_valid_new["history"].apply(transform_text_to_list)
# df_valid_new.head(3)
df_valid_new = df_valid_new.explode("history", ignore_index=True)
df_valid_new.head(3)

In [ ]:
df_valid_new.loc[0,"history"]

In [ ]:
df_valid_exploded = pd.concat([df_ratings, df_valid_new])
df_valid_exploded

In [ ]:
# duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['userId','history'], keep=False)]
# duplicates = duplicates[duplicates["score"].isna()]
duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['userId','history'], keep='first')]
duplicates
# duplicates = df_valid_exploded[df_valid_exploded.duplicated(subset=['history'], keep=False)]
# duplicates[duplicates["history"]=='253339a1-92b6-44d1-8fa2-59236c5251b1']
# duplicates[duplicates["userId"]=='f0e758359a184c99912e1ad5fc912b92b5e7b63c7a6002']

In [ ]:
df_valid_cleaned = df_valid_exploded.drop_duplicates(subset=['userId','history'], keep="first")
df_valid_cleaned

In [ ]:
df_valid_cleaned["score"].fillna(0,inplace=True)
df_valid_cleaned

In [ ]:
dataset_valid = Dataset()

# Get unique values for users, items, and user features
unique_users_valid = df_valid_cleaned["userId"].unique()
unique_items_valid = df_valid_cleaned["history"].unique()
unique_user_features_valid = df_valid_cleaned["userType"].unique().tolist()

# Fit dataset with users, items, and user feature names
dataset_valid.fit_partial(
    users=unique_users_valid,
    items=unique_items_valid,
    user_features=unique_user_features_valid  # Register user features
)

In [ ]:
(interactions_valid, weights_valid) = dataset_valid.build_interactions([
    (row.userId, row.history, row.score) 
    for _, row in df_valid_cleaned.iterrows()
])

In [ ]:
print(unique_users_valid.shape)
print(unique_items_valid.shape)
print(len(unique_user_features_valid))

In [ ]:
interactions_valid.shape

In [ ]:
user_features_list_valid = [
    (row.userId, [row.userType])  
    for _, row in df_valid.iterrows()
]

user_features_valid = dataset_valid.build_user_features(user_features_list_valid)

In [ ]:
# print(user_features_valid)

In [ ]:
import pickle

loaded_model = pickle.load(open('artifacts/lightfm_model.pkl', 'rb'))
loaded_user_id_map = pickle.load(open('artifacts/user_id_map.pkl', 'rb'))
loaded_item_id_map_reverse = pickle.load(open('artifacts/item_id_map_reverse.pkl', 'rb'))
loaded_user_feature_map = pickle.load(open('artifacts/user_feature_map.pkl', 'rb'))

In [ ]:
loaded_model.fit_partial(interactions=interactions_valid, sample_weight=weights_valid, user_features=user_features_valid,
                         epochs=NO_EPOCHS, num_threads=NUM_THREADS)
# loaded_model.fit_partial(interactions=interactions_valid, sample_weight=weights_valid,
#                          epochs=NO_EPOCHS, num_threads=NUM_THREADS)

In [37]:
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset_valid.mapping()
item_id_map_reverse = {v: k for k, v in item_id_map.items()}

loaded_user_id_map = user_id_map
loaded_item_id_map_reverse = item_id_map_reverse
loaded_user_feature_map = user_feature_map

In [38]:
# _, n_items = interactions.shape # no of users * no of items
_, n_items_valid = interactions_valid.shape # no of users * no of items

# n_items = NUM_ITEMS_TRAIN
n_items = n_items_valid

### Make predictions to known and unknowm on same recommendation function with pkls

In [39]:
def format_newuser_input(user_feature_map, user_feature_list):
  normalised_val = 1.0 
  target_indices = []
  for feature in user_feature_list:
    try:
        target_indices.append(user_feature_map[feature])
    except KeyError:
        # print("new user feature encountered '{}'".format(feature))
        pass
  #print("target indices: {}".format(target_indices))
  new_user_features = np.zeros(len(user_feature_map.keys()))
  for i in target_indices:
    new_user_features[i] = normalised_val
  new_user_features = sparse.csr_matrix(new_user_features)
  return(new_user_features)

In [40]:
def sample_recommendation_by_title(user_hash,df_news,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:10]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    print("Top 5 recommended items:")

    for x in top_5_items:
        row = df_news[df_news["page"] == x]
        print("        %s" % row["title"].values[0])


In [41]:
# predict for known user
# df_valid["userId"].iloc[0]
user_feature_list = [df_valid["userType"].iloc[0]]
user_hash = df_valid["userId"].iloc[0]

# sample_recommendation_by_title(user_hash,df_news,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)

In [42]:
def get_recommended_history_list(user_hash,user_feature_list,item_id_map_reverse,user_feature_map,user_id_map,model):
    """
    This function verifies if the users is known or new, and makes recommendations depending on this verification.
    The top 5 recommendations from the list are returned.
    """
    try:
        user_x = user_id_map[user_hash]
        scores = model.predict(user_x, np.arange(n_items)) # means predict for all
    except:
        new_user_features = format_newuser_input(user_feature_map, user_feature_list)
        # scores = model.predict(0, np.arange(n_items))
        scores = model.predict(0, np.arange(n_items), user_features=new_user_features)
    
    top_5_indices = np.argsort(-scores)[:20]  # Sort scores in descending order and take the top 5
    top_5_items = [item_id_map_reverse[i] for i in top_5_indices]

    return top_5_items

In [43]:
from utils.custom_treat_data_funcs import transform_text_to_list

# Transform the single string with histories/items into a list of strings

df_valid["history"] = df_valid["history"].apply(transform_text_to_list)
df_valid

AttributeError: 'list' object has no attribute 'replace'

In [44]:
# Testing for just one user

user_feature_list = [df_valid["userType"].iloc[0]]
user_hash = df_valid["userId"].iloc[0]
validation_history_hashes = df_valid["history"].iloc[0]

recommeded_histories = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
print(recommeded_histories)
print(validation_history_hashes)

['d2593c3d-2347-40d9-948c-b6065e8459a9', 'f6b5d170-48b9-4f8e-88d4-c84b6668f3bd', '1f32787b-de2b-49be-8c20-ddaeae34cc22', '6a83890a-d9e9-4f6b-a6c6-90d031785bbf', 'f0a78e58-ec7e-494c-9462-fbd6446a9a89', '4c63d7cd-4902-4ffb-9b94-578b1b2151f0', '855d20b7-53f2-4678-a10f-55402d085018', 'bf257382-74fb-4392-ad6a-143240e39f81', '1c27cf97-b20c-4e40-b1f1-288b721517b3', 'a36c98b5-f159-48f8-9f5a-1fc6ea9956c8', '4e9c2825-ff13-41ca-8e91-edd848060d19', '882e7c95-935a-4eab-9ece-f85f5f7d0f4e', '15281e10-e6bc-48bc-9b1b-94402f83699b', '8c246d2b-81bd-4c1f-b563-2c905675f984', '89fa73f0-4341-4de4-bb2a-e429ef96bd43', '29b6b142-4173-4ec4-832f-7d0a32255c10', '7b056bf6-c232-46bd-8903-59145ff7ce46', '4700f517-5c5d-483c-81b7-d77aca04991c', 'e384ec29-136e-4241-9321-49b367b8cbd5', '458bf0ec-efb4-4bfd-9446-c80295e6aa87']
['be89a7da-d9fa-49d4-9fdc-388c27a15bc8', '01c59ff6-fb82-4258-918f-2910cb2d4c52']


In [45]:
def count_valid_recommendations(validation_history_hashes, recommeded_histories):
    """
    This function receives 
    * The `validation_history_hashes` (which is a list of histories contained in the "validacao.csv")
    * The `recommeded_histories` (which are the recommended histories/items by the model)
    Then, it verifies how many recommended items match with the validation histories.
    """
    count_valids = 0
    for valid_hist in validation_history_hashes:
        if valid_hist in recommeded_histories:
            count_valids = count_valids+1
    return count_valids

In [46]:
df_valid["recommended_hists"] = "" # creates a column to receive the list of recommended histories
df_valid["matched_recommendations"] = 0 # creates a column to receive the quantity of matches between recommendations and validation items
df_valid["historySize"] = df_valid["history"].apply(lambda x : len(x)) # calculate the quantity of histories for each user

In [47]:
df_valid.head(2)

,userId,userType,history,timestampHistory,recommended_hists,matched_recommendations,historySize
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,"[be89a7da-d9fa-49d4-9fdc-388c27a15bc8, 01c59ff...",[1660533136590 1660672113513],,0,2
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,[77901133-aee7-4f7b-afc0-652231d76fe9],[1660556860253],,0,1


In [48]:
import multiprocessing

def parallelize_dataframe(df, func):
    num_cores = multiprocessing.cpu_count()-1  #leave one free to not freeze machine
    num_partitions = num_cores #number of partitions to split dataframe
    df_split = np.array_split(df, num_partitions)
    pool = multiprocessing.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [49]:
# def validate_recommendations(df_valid):
for _, row in df_valid.iterrows():
    """
    For each user, get the top 5 recommendations from the model, and also count how many of them are contained within the validation set.
    Stores the results on the columns `recommended_hists` `matched_recommendations`.
    """
    user_feature_list = [row["userType"]]
    user_hash = row["userId"]
    validation_history_hashes = row["history"]

    recommended_hist = get_recommended_history_list(user_hash,user_feature_list,loaded_item_id_map_reverse,loaded_user_feature_map,loaded_user_id_map,loaded_model)
    num_valid_recommendations = count_valid_recommendations(validation_history_hashes, recommended_hist)

    row["recommended_hists"] = recommended_hist
    row["matched_recommendations"] = num_valid_recommendations
        

In [50]:
df_valid.head(2)

,userId,userType,history,timestampHistory,recommended_hists,matched_recommendations,historySize
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,"[be89a7da-d9fa-49d4-9fdc-388c27a15bc8, 01c59ff...",[1660533136590 1660672113513],,0,2
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,[77901133-aee7-4f7b-afc0-652231d76fe9],[1660556860253],,0,1


In [51]:
df_valid.describe()

,matched_recommendations,historySize
count,10000.0,10000.00000
mean,0.0,1.90410
std,0.0,1.23962
min,0.0,1.00000
25%,0.0,1.00000
50%,0.0,1.00000
75%,0.0,2.00000
max,0.0,5.00000
